Preparation of dataframes just before data analysis <br>
QUICK information about following datasets: <br>
* diagnoses_phecode_list_df <br>
* diagnoses_ICD10_df <br>
* prescriptions_df <br>
* prescriptions_ATC4_list_df <br>
* episodes : before merging the episodes <br>
* full_episodes : after merging the episodes <br>
* final_episodes : removing some outliers, and adding all the calculated features not the ones related to diagnoses and medications 
episodes <br>
* fullHot_episodes : All the calculated features PLUS the ones related to diagnoses and medications and some more <br>
* ######################
* Full_Phecode_ATC4 : merging of fullHot_episodes and actual diagnoses(Phecodes) and medications(ATC4)  <br>
* Full_Phecode_ATC : merging of fullHot_episodes and actual diagnoses(Phecodes) and medications(ATC)  <br>
* Full_ICD10_ATC : merging of fullHot_episodes and actual diagnoses(ICD10) and medications(ATC) <br>

In [1]:
import pandas as pd
import numpy as np

In [18]:
diagnoses_phecode_list_df = pd.read_csv("/home/kabank/workbench/.conda/analysis/kabank-data/diagnoses_listed.csv")
diagnoses_phecode_list_df['episode_id'] = diagnoses_phecode_list_df['episode_id'].astype(int)
print(diagnoses_phecode_list_df.head(3))
print(len(diagnoses_phecode_list_df)) #16,390 episodes

   episode_id                diagnoses
0          10  ['3009', '313', '3131']
1       10001          ['2972', '969']
2       10002                 ['3131']
16390


In [25]:
diagnoses_ICD10_df = pd.read_csv( "/home/kabank/workbench/.conda/analysis/Resources/Diagnosis.CSV", decimal=',')
# Remove duplicates from diagnoses_ICD10_df
print(len(diagnoses_ICD10_df))
diagnoses_ICD10_df['episode_id'] = diagnoses_ICD10_df['episode_id'].astype(float).astype(int)

diagnoses_ICD10_list_df = diagnoses_ICD10_df.groupby('episode_id').agg({
    'diagnosis': lambda x: list(set(x))
})
diagnoses_ICD10_list_df.rename(columns={'diagnosis': 'diagnosis_ICD10'}, inplace=True)
diagnoses_ICD10_list_df = diagnoses_ICD10_list_df.reset_index()

print(diagnoses_ICD10_list_df.head(3))
print(len(diagnoses_ICD10_list_df))
print(diagnoses_ICD10_list_df['episode_id'].nunique())

42803
   episode_id diagnosis_ICD10
0           4    [F900, G479]
1           5          [H811]
2           6          [F931]
16552
16552


In [26]:
# Prescriptions
#Sql query for extracting prescriptions 
prescriptions_df = pd.read_csv( "/home/kabank/workbench/.conda/analysis/kabank-data/prescriptionsEachSak.csv")
prescriptions_df.dropna(subset=['episode_id', ], inplace=True)
prescriptions_df['episode_id'] = prescriptions_df['episode_id'].astype(int)
print(len(prescriptions_df)) #44685 episodes

prescriptions_ATC_list_df = prescriptions_df.groupby('episode_id').agg({
    'atc_code': lambda x: list(set(x))
})
prescriptions_ATC_list_df = prescriptions_ATC_list_df.reset_index()
#print(len(prescriptions_ATC_list_df))# 4463 episodes
print(prescriptions_ATC_list_df.head(3))
print(len(prescriptions_ATC_list_df))
print(prescriptions_ATC_list_df['episode_id'].nunique())

44685
   episode_id                              atc_code
0           9                    [N06AB03, N06AB06]
1          27  [N06BA04, N06AB06, N05CH01, R06AD01]
2          28                    [N06AB03, N06AB06]
4463
4463


In [28]:
# Prescriptions
#Sql query for extracting prescriptions had 56,167 rows
prescriptions_df2 = pd.read_csv( "/home/kabank/workbench/.conda/analysis/kabank-data/prescriptionsEachSak.csv")
prescriptions_df2 = prescriptions_df2.dropna(subset=['episode_id'])
prescriptions_df2['ATC4'] = prescriptions_df2['atc_code'].str[:4]
prescriptions_df2['episode_id'] = prescriptions_df2['episode_id'].astype(int)
prescriptions_df2.head(3)
prescriptions_ATC4_list_df = prescriptions_df2.groupby('episode_id').agg({
    'ATC4': lambda x: list(set(x))
})

prescriptions_ATC4_list_df = prescriptions_ATC4_list_df.reset_index()

#print(len(prescriptions_ATC4_list_df))# 4463 episodes
print(prescriptions_ATC4_list_df.head(3))
print(len(prescriptions_ATC4_list_df))
print(prescriptions_ATC4_list_df['episode_id'].nunique())

   episode_id                      ATC4
0           9                    [N06A]
1          27  [N05C, R06A, N06A, N06B]
2          28                    [N06A]
4463
4463


In [7]:
episodes = pd.read_csv('/home/kabank/workbench/.conda/analysis/Resources/epiosdes2.csv') # before merge 18646
full_episodes = pd.read_csv('/home/kabank/workbench/.conda/analysis/Resources/full_episodes.csv') # after merge 17433
full_episodes.to_csv('/home/kabank/workbench/.conda/analysis/kabank-data/new-data/full_episodes.csv', index=False)
print('Columns of episodes:\n', episodes.columns, '\n len:', len(episodes))
print('\n')
print('Columns of full_episodes:\n', full_episodes.columns, '\n len:', len(full_episodes))

Columns of episodes:
 Index(['pasient', 'episode_id', 'patient_age', 'remaining_time_countdown',
       'var_no_dates_permonth', 'gender', 'episode_order', 'islast',
       'closingcode', 'aftercode', 'instanskode', 'episode_start_date',
       'episode_end_date', 'length_of_episode', 'tillnextepisode',
       'starttillnextstart', 'count_poliklinikk', 'count_familieavdeling',
       'count_dagavdeling', 'count_dognavdeling', 'count_osv',
       'count_outpatient', 'count_inpatient', 'count_lmssciadm',
       'count_inpatient_day', 'count_inpatient_daynight', 'count_terapi',
       'count_undersokelse', 'count_radge', 'count_behplanlegging',
       'count_ikkemott', 'count_others'],
      dtype='object') 
 len: 24461


Columns of full_episodes:
 Index(['Unnamed: 0', 'pasient', 'episode_id', 'age',
       'remaining_time_countdown', 'var_no_dates_permonth', 'gender',
       'episode_order', 'islast', 'closingcode', 'aftercode', 'instanskode',
       'episode_start_date', 'episode_end_da

In [29]:
# We removed some outliers, this is containing all the features not the ones related to diagnoses and medications like 'num_diagnoses', 'num_medications': 17,125 episodes
final_episodes = pd.read_csv("/home/kabank/workbench/.conda/analysis/kabank-data/final_episodes.csv")
fullHot_episodes = pd.read_csv("/home/kabank/workbench/.conda/analysis/kabank-data/final_episodes4.csv") 
print(final_episodes['gender'].value_counts())
print(final_episodes.shape)
print(fullHot_episodes.shape)
final_episodes = pd.merge(final_episodes, fullHot_episodes[['episode_id', 'num_diagnoses', 'num_medications']], on='episode_id', how='inner')
print(final_episodes.shape)

final_episodes['gender'] = final_episodes['gender'].replace({'M': 0, 'F': 1, np.nan: 2}).astype(int)

final_episodes.loc[final_episodes["age"].between(0, 5), ["age_group"]] = "Preschooler"
final_episodes.loc[final_episodes["age"].between(6, 11), ["age_group"]] = "MiddleChildhood"
final_episodes.loc[final_episodes["age"].between(12, 18), ["age_group"]] = "Teenager" 
final_episodes.loc[final_episodes["age"].between(19, 60), ["age_group"]] = "Adult"

fullHot_episodes.loc[fullHot_episodes["age"].between(0, 5), ["age_group"]] = "Preschooler"
fullHot_episodes.loc[fullHot_episodes["age"].between(6, 11), ["age_group"]] = "MiddleChildhood"
fullHot_episodes.loc[fullHot_episodes["age"].between(12, 18), ["age_group"]] = "Teenager" 
fullHot_episodes.loc[fullHot_episodes["age"].between(19, 60), ["age_group"]] = "Adult"

fullHot_episodes.to_csv('/home/kabank/workbench/.conda/analysis/kabank-data/new-data/fullHot_episodes.csv', index=False)
final_episodes.to_csv('/home/kabank/workbench/.conda/analysis/kabank-data/new-data/final_episodes.csv', index=False)
print('Number of rows in final_episodes:\n',len(final_episodes))
print('Columns of final_episodes:\n', final_episodes.columns)
print('\n')
print('Number of rows in fullHot_episodes:\n',len(fullHot_episodes))
print('Columns of fullHot_episodes:\n', fullHot_episodes.columns)
fullHot_episodes['Count_visit'].describe()

gender
M    11898
F    10731
0       47
Name: count, dtype: int64
(22676, 38)
(22676, 59)
(22676, 40)
Number of rows in final_episodes:
 22676
Columns of final_episodes:
 Index(['Unnamed: 0', 'pasient', 'episode_id', 'age',
       'remaining_time_countdown', 'var_no_dates_permonth', 'gender',
       'episode_order', 'islast', 'closingcode', 'aftercode',
       'episode_start_date', 'episode_end_date', 'tillnextepisode',
       'Length_of_Episode', 'Cat_LOE', 'Cat_TNE', 'TNE_BO_180', 'TNE_NO_180',
       'TNE_BO_365', 'TNE_NO_365', 'TNE_BO_730', 'TNE_NO_730', 'TNE_BO_1095',
       'TNE_NO_1095', 'Cat_LOE_desc', 'Count_visit', 'Cat_CV', 'Therapy_ratio',
       'Examination_ratio', 'Advisory_ratio', 'TreatmentPlanning_ratio',
       'Outpatient_ratio', 'Inpatient_ratio', 'Inpatient_day_ratio',
       'Inpatient_daynight_ratio', 'Care_intensity', 'age_group',
       'num_diagnoses', 'num_medications'],
      dtype='object')


Number of rows in fullHot_episodes:
 22676
Columns of fullHot_ep

count    22676.000000
mean        89.416431
std        134.491413
min          0.000000
25%         16.000000
50%         50.000000
75%        110.000000
max       4159.000000
Name: Count_visit, dtype: float64

In [39]:
# Merging of fullHot_episodes and ICD10 and medications(ATC4)

print(fullHot_episodes.shape)
print(diagnoses_ICD10_list_df.shape)

fullHot_episodes = pd.read_csv("/home/kabank/workbench/.conda/analysis/kabank-data/new-data/fullHot_episodes.csv") 
Full_ICD10 = pd.merge(fullHot_episodes, diagnoses_ICD10_list_df[['episode_id', 'diagnosis_ICD10']], on='episode_id', how='left')

print(Full_ICD10.shape)
print(prescriptions_ATC4_list_df.shape)

Full_ICD10_ATC4 = pd.merge(Full_ICD10, prescriptions_ATC4_list_df[['episode_id', 'ATC4']], on='episode_id', how='left')

print(Full_ICD10_ATC4.shape)
Full_ICD10_ATC4.rename(columns={'diagnosis_ICD10': 'actual_diag_ICD10'}, inplace=True)
Full_ICD10_ATC4.rename(columns={'ATC4': 'actual_med_ATC4'}, inplace=True)

print(len(Full_ICD10_ATC4))
# Full_ICD10_ATC4[['episode_id', 'actual_diag_ICD10', 'actual_med_ATC4']]
# Full_ICD10_ATC4.columns
# prescriptions_ATC4_list_df.head(10)
# Full_ICD10_ATC4.head()

Full_ICD10_ATC4.to_csv('/home/kabank/workbench/.conda/analysis/kabank-data/new-data/Full_ICD10_ATC4.csv', index=False)




(22676, 59)
(16552, 2)
(22676, 60)
(4463, 2)
(22676, 61)
22676


In [43]:
# it looks like merging of fullHot_episodes and actual diagnoses(Phecodes) and medications(ATC4)

print(fullHot_episodes.shape)
print(diagnoses_phecode_list_df.shape)
Full_Phecode = pd.merge(fullHot_episodes, diagnoses_phecode_list_df[['episode_id', 'diagnoses']], on='episode_id', how='left')
print(Full_Phecode.shape)
print(prescriptions_ATC4_list_df.shape)
Full_Phecode_ATC4 = pd.merge(Full_Phecode, prescriptions_ATC4_list_df[['episode_id', 'ATC4']], on='episode_id', how='left')
print(Full_Phecode_ATC4.shape)
Full_Phecode_ATC4.rename(columns={'diagnoses': 'actual_diag_Phe'}, inplace=True)
Full_Phecode_ATC4.rename(columns={'ATC4': 'actual_med_ATC4'}, inplace=True)

print(len(Full_Phecode_ATC4))
# Full_Phecode_ATC4.columns
# Full_Phecode_ATC4.head(30)

Full_Phecode_ATC4.to_csv('/home/kabank/workbench/.conda/analysis/kabank-data/new-data/Full_Phecode_ATC4.csv', index=False)


(22676, 59)
(16390, 2)
(22676, 60)
(4463, 2)
(22676, 61)
22676


In [48]:
# it looks like merging of fullHot_episodes and actual diagnoses(Phecodes) and medications(Full ATC)

print(fullHot_episodes.shape)
print(diagnoses_phecode_list_df.shape)
Full_Phecode = pd.merge(fullHot_episodes, diagnoses_phecode_list_df[['episode_id', 'diagnoses']], on='episode_id', how='left')
print(Full_Phecode.shape)
print(prescriptions_ATC_list_df.shape)
Full_Phecode_ATC = pd.merge(Full_Phecode, prescriptions_ATC_list_df[['episode_id', 'atc_code']], on='episode_id', how='left')
Full_Phecode_ATC.rename(columns={'diagnoses': 'actual_diag_Phe'}, inplace=True)
Full_Phecode_ATC.rename(columns={'atc_code': 'actual_med_Full_ATC'}, inplace=True)

print(len(Full_Phecode_ATC))
#Full_Phecode_ATC[['actual_diag_Phe', 'actual_med_Full_ATC']]

Full_Phecode_ATC.to_csv('/home/kabank/workbench/.conda/analysis/kabank-data/new-data/Full_Phecode_ATC.csv', index=False)
#Full_Phecode_ATC.head(30)

(22676, 59)
(16390, 2)
(22676, 60)
(4463, 2)
22676


In [51]:
# it looks like merging of fullHot_episodes and actual diagnoses(ICD10) and medications(Full ATC)
print(fullHot_episodes.shape)
print(diagnoses_ICD10_list_df.shape)

Full_ICD10 = pd.merge(fullHot_episodes, diagnoses_ICD10_list_df, on='episode_id', how='left')
print(Full_ICD10.shape)
print(prescriptions_ATC_list_df.shape)
Full_ICD10_ATC = pd.merge(Full_ICD10, prescriptions_ATC_list_df[['episode_id', 'atc_code']], on='episode_id', how='left')
print(Full_ICD10_ATC.shape)
Full_ICD10_ATC.rename(columns={'diagnosis_ICD10': 'actual_diag_ICD10'}, inplace=True)
Full_ICD10_ATC.rename(columns={'atc_code': 'actual_med_Full_ATC'}, inplace=True)
Full_ICD10_ATC.to_csv('/home/kabank/workbench/.conda/analysis/kabank-data/new-data/Full_ICD10_ATC.csv', index=False)
print(len(Full_ICD10_ATC))
# Full_ICD10_ATC.columns
# Full_ICD10_ATC.head()

(22676, 59)
(16552, 2)
(22676, 60)
(4463, 2)
(22676, 61)
22676
